In [1]:
from datetime import datetime
from pydantic import BaseModel, field_validator, ValidationError

In [2]:
class Model(BaseModel):
    dt: datetime

In [4]:
Model(dt="2020-10-01T12:00:00Z")

Model(dt=datetime.datetime(2020, 10, 1, 12, 0, tzinfo=TzInfo(UTC)))

In [6]:
try:
    Model(dt="2020/10/01 03:00pm")
except ValidationError as e:
    print(e)

1 validation error for Model
dt
  Input should be a valid datetime or date, invalid date separator, expected `-` [type=datetime_from_date_parsing, input_value='2020/10/01 03:00pm', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/datetime_from_date_parsing


In [7]:
try:
    Model(dt="Jan 1, 2020 03:00pm")
except ValidationError as e:
    print(e)

1 validation error for Model
dt
  Input should be a valid datetime or date, invalid character in year [type=datetime_from_date_parsing, input_value='Jan 1, 2020 03:00pm', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/datetime_from_date_parsing


In [8]:
from dateutil.parser import parse

In [9]:
parse("Jan 1, 2020 03:00pm")

datetime.datetime(2020, 1, 1, 15, 0)

In [10]:
parse("2020/10/01 03:00pm")


datetime.datetime(2020, 10, 1, 15, 0)

In [11]:
try:
    parse(datetime(2020, 10, 1, 3, 0, 0))
except TypeError as e:
    print(e)

Parser must be a string or character stream, not datetime


In [12]:
Model(dt=datetime(2020, 10, 1, 3, 0, 0))

Model(dt=datetime.datetime(2020, 10, 1, 3, 0))

In [13]:
from typing import Any

class Model(BaseModel):
    dt: datetime

    @field_validator("dt", mode="before")
    @classmethod
    def parse_datetime(cls, v: Any) -> datetime:
        if isinstance(v, str):
            print(f"Parsing string {v}")
            try:
                return parse(v)
            except Exception as e:
                raise ValueError(str(e))
        print("pass through...")
        return v



In [14]:
Model(dt="Jan 1, 2020 03:00pm")

Parsing string Jan 1, 2020 03:00pm


Model(dt=datetime.datetime(2020, 1, 1, 15, 0))

In [15]:
Model(dt=datetime(2020, 10, 1, 3, 0, 0))

pass through...


Model(dt=datetime.datetime(2020, 10, 1, 3, 0))

In [16]:
try:
    Model(dt=[1, 2, 3])
except ValidationError as e:
    print(e)

pass through...
1 validation error for Model
dt
  Input should be a valid datetime [type=datetime_type, input_value=[1, 2, 3], input_type=list]
    For further information visit https://errors.pydantic.dev/2.9/v/datetime_type


In [17]:
class Model(BaseModel):
    dt: datetime

    @field_validator("dt", mode="before")
    @classmethod
    def parse_datetime(cls, v: Any) -> datetime:
        if isinstance(v, str):
            print(f"Parsing string {v}")
            try:
                return parse(v)
            except Exception as e:
                raise ValueError(str(e))
        print("pass through...")
        return v

    @field_validator("dt", mode="before")
    @classmethod
    def validate_1(cls, v: Any) -> datetime:
        print(f"Validating {v} in validate_1")
        return v

    @field_validator("dt", mode="before")
    @classmethod
    def validate_2(cls, v: Any) -> datetime:
        print(f"Validating {v} in validate_2")
        return v

In [18]:
Model(dt=datetime(2020, 10, 1, 3, 0, 0))

Validating 2020-10-01 03:00:00 in validate_2
Validating 2020-10-01 03:00:00 in validate_1
pass through...


Model(dt=datetime.datetime(2020, 10, 1, 3, 0))